In [5]:
!pip install groq mcp

In [6]:
import getpass
import os
print("Enter your Groq API")
GROQ_API_KEY=getpass.getpass()
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

Enter your Groq API
··········


In [13]:
# 1. Install necessary libraries
!pip install mcp groq nest_asyncio -q

import os
import json
import threading
import time
import random
import nest_asyncio
from mcp.server.fastmcp import FastMCP
from groq import Groq
import getpass

# Apply nest_asyncio to allow async operations in Colab
nest_asyncio.apply()

# --- STEP 1: SETUP YOUR API KEY ---
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
if GROQ_API_KEY == "YOUR_GROQ_API_KEY":
    from google.colab import userdata
    try:
        GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    except:
        print("Please enter your Groq API Key: ")
        GROQ_API_KEY = getpass.getpass()

# --- STEP 2: DUMMY DATABASE ---
class_data = [
    {"name": "Alice Smith", "marks": 85, "attendance": "92%", "notes": "Consistent performer."},
    {"name": "Bob Johnson", "marks": 42, "attendance": "60%", "notes": "Needs improvement in Math."},
    {"name": "Charlie Brown", "marks": 78, "attendance": "88%", "notes": "Active participant."},
    {"name": "Diana Prince", "marks": 95, "attendance": "98%", "notes": "Top of the class."},
    {"name": "Ethan Hunt", "marks": 70, "attendance": "75%", "notes": "Often misses morning sessions."}
]

# --- STEP 3: DEFINE THE MCP SERVER ---
mcp = FastMCP("ClassroomServer")

@mcp.tool()
def get_student_details(name: str) -> str:
    """Get marks, attendance, and notes for a specific student by name."""
    for student in class_data:
        if name.lower() in student["name"].lower():
            return json.dumps(student)
    return "Error: Student not found in the database."

@mcp.tool()
def list_all_students() -> str:
    """Returns a list of all students currently in the classroom database."""
    names = [s["name"] for s in class_data]
    return json.dumps({"students": names})

# --- STEP 4: RUN THE SERVER (With Port Fix) ---
# We use a random port so that re-running the cell doesn't cause "Address already in use"
PORT = random.randint(8001, 8999)

def run_mcp_server():
    try:
        mcp.run(transport="streamable-http", port=PORT)
    except:
        pass # Ignore errors if server is already cleaning up

server_thread = threading.Thread(target=run_mcp_server, daemon=True)
server_thread.start()
print(f"✅ MCP Server is starting on port {PORT}...")
time.sleep(2)

# --- STEP 5: GROQ AI AGENT LOGIC ---
client = Groq(api_key=GROQ_API_KEY)

def ask_ai(user_query):
    # SYSTEM PROMPT: This is the most important part to stop the AI from hallucinating
    messages = [
        {
            "role": "system",
            "content": "You are a helpful classroom assistant. You MUST use the provided tools to answer questions about students. If the tool returns 'Student not found', tell the user that. Do not make up dummy names or data."
        },
        {"role": "user", "content": user_query}
    ]

    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_student_details",
                "description": "Retrieve the marks, attendance percentage, and any specific notes for a student. Use this when the user asks for details about a single student's performance or information.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string", "description": "The partial of full name of the student to retrieve details for (e.g., 'Alice Smith', 'Bob Johnson')."}
                    },
                    "required": ["name"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "list_all_students",
                "description": "Provides a complete list of all student names currently enrolled in the class. Use this when the user asks for who is in the class, a list of students, or to identify available students."
            }
        }
    ]

    # 1. First request to Groq
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant", # Using a larger model for better tool following
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # 2. Process tool calls
    if tool_calls:
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"🛠️  [MCP Tool Used]: {function_name}({args})")

            if function_name == "get_student_details":
                result = get_student_details(args.get("name"))
            elif function_name == "list_all_students":
                result = list_all_students()

            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": result
            })

        # 3. Get the final response based on tool results
        final_response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=messages
        )
        return final_response.choices[0].message.content

    return response_message.content

# --- STEP 6: INTERACTIVE CHAT ---
print("\n--- Classroom Assistant Ready ---")
print("Ask about your students (e.g., 'Who is in my class?', 'How is Alice doing?')")
print("Type 'exit' to stop.")

while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ['exit', 'quit']:
        break

    try:
        response = ask_ai(user_input)
        print(f"AI: {response}")
    except Exception as e:
        print(f"Error: {e}")

✅ MCP Server is starting on port 8665...

--- Classroom Assistant Ready ---
Ask about your students (e.g., 'Who is in my class?', 'How is Alice doing?')
Type 'exit' to stop.

You: who is in my class?
🛠️  [MCP Tool Used]: list_all_students(None)
AI: Unfortunately, I cannot access or provide any information about individuals outside of the provided data. 

Based on the information I've been given, the students in your class are:

1. Alice Smith
2. Bob Johnson
3. Charlie Brown
4. Diana Prince
5. Ethan Hunt

You: how is ethan doing?
🛠️  [MCP Tool Used]: get_student_details({'name': 'Ethan'})
AI: It looks like Ethan is doing okay, academically. His marks are around 70 and his attendance is about 75%. However, his notes indicate that he often misses the morning sessions, which might be something to keep an eye on.

You: how is alice doing?
🛠️  [MCP Tool Used]: get_student_details({'name': 'Alice Smith'})
AI: Alice Smith appears to be doing well. She has achieved an overall score of 85, atten